Import all our necessary libraries.

In [1]:
import json
import datetime
import inspect
import requests
from dotenv import load_dotenv
from typing import (TypedDict, List, Dict, Literal, Callable, Optional, Any, get_type_hints)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall

In [2]:
load_dotenv()
client = OpenAI()

Insert our Memory class object. We built this class in order to have a standardized memory function object that will behave across all our AI use cases. This use case being a ReAct agent.

In [3]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(self, 
        role: Literal['user', 'system', 'assistant', 'tool'], 
        content: str,
        tool_calls: dict=dict(),
        tool_call_id=None)->None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)
    
    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages
    
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

Insert our Tool class object. Similar to how we created our memory class, we created a Tool class object so that we could leverage the framework and its behavior repeatedly in multiple different use cases. This use case being a ReAct Agent.

class below has been updated and is the best version of this object i have

In [6]:
class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.description = func.__doc__
        self.argument_types_map = get_type_hints(func)
        self.signature = inspect.signature(func)
        self.arguments = [
            {
                "name": key,
                "type":self._infer_json_schema_type(value),
                "required": param.default == inspect.Parameter.empty
            }
            for key, value in self.argument_types_map.items()
            if (param := self.signature.parameters.get(key))
        ]

    def dict(self):
        return{
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"]
                        for argument in self.arguments
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True
            }
        }

    def __call__ (self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def _infer_json_schema_type(self, arg_type: Any) -> str:
        if arg_type == bool:
            return "boolean"
        elif arg_type == int:
            return "integer"
        elif arg_type == float:
            return "number"
        elif arg_type == str:
            return "string"
        elif arg_type == list:
            return "array"
        elif arg_type == dict:
            return "object"
        elif arg_type == None:
            return "Null"
        elif arg_type == datetime.date or arg_type == datetime.datetime:
            return "string"
        else:
            return "string"


In this use case, we will build a ReAct Tool loop. in order to create a usable loop, we need the loop to stop at some point.

In [7]:
class StopReactLoopException(Exception):
    """
    Terminates React Loop.
    """

In [8]:
TERMINATION_MESSAGE = "StopReactLoopException"

Next, we need a function that we'll be using as a tool so the LLM knows when to terminate the loop. once we have that function, we can continue to work on our agent class.

In [9]:
def termination()-> str:
    """Terminate the ReAct Loop. If the agent thinks theres no further action to take"""
    return TERMINATION_MESSAGE

Now lets build our Agent Class 

The first def __init__ is the python class constructor. This function is the special method that gets called when we create a new object from a class. It's main job is to initialize the new object's state

In [16]:
class Agent:
    """A ReAct AI Agent"""

    def __init__(
        self,
        name:str = "Agent",
        role:str = "Personal Assistant",
        instructions:str = "Help users with any questions",
        model:str = "gpt-4o-mini",
        temperature:float = 0.1,
        tools:List[Tool] = [],
        ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.client = OpenAI()
        self.tools = tools
        self.termination_message = TERMINATION_MESSAGE
        self._register_tool(Tool(termination))
        self.tool_map = {t.name:t for t in tools}
        self.openai_tools = [t.dict() for t in self.tools] if self.tools else None
        self.memory = Memory()

        self.memory.add_message(
            role="system",
            content=f"You're an AI agent, your role is {self.role}, "
                    f"and you need to {self.instructions} "
                    "You can answer multistep questions by sequentially calling functions. "
                    "You follow a pattern of thought and action. "
                    "Create a plan of execution: "
                    "- Use thought to describe your thoughts about the question you've been asked. "
                    "- Use action to specify one of the tools available to you. "
                    "When you think its over, call the termination tool. "
                    "Never try to respond directly if the question requires a tool."
                    "The actions you have are the tools: "
                    f"{self.openai_tools}",
        )

    def invoke(self, user_message: str, max_iter:int=3) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )
        try:
            self._react_loop(max_iter)
        except StopReactLoopException as e:
            print(f"Terminated loop with message: '{e}'")
            self._reason()

        return self.memory.last_message()
    
    def _react_loop(self, max_iter:int):
        for i in range(max_iter):
            self._reason()

            ai_message = self._get_completion(
                messages = self.memory.get_messages(),
                tools=self.openai_tools,
            )
            tool_calls = ai_message.tool_calls

            self.memory.add_message(
                role="assistant",
                content=ai_message.content,
                tool_calls=tool_calls,
            )

            if tool_calls:
                self._call_tools(tool_calls)
            
    def _reason(self):
        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=None,
        )

    def _call_tools(self, tool_calls:List[ChatCompletionMessageToolCall]):
        for t in tool_calls:
            tool_call_id = t.id
            function_name = t.function.name
            args = json.loads(t.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool",
                content=str(result),
                tool_call_id=tool_call_id,
            )
            if result == TERMINATION_MESSAGE:
                raise StopReactLoopException(TERMINATION_MESSAGE)
    
    def _get_completion(self, messages:List[Dict], tools:List=None)-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=tools,
        )
        
        return response.choices[0].message
    
    def _register_tool(self, tool:Tool):
        self.tools.append(tool)


In [17]:
def power(base:float, exponent:float):
    """Exponentiation: base to the power of exponent."""

    return base ** exponent

In [18]:
def sum(num_1:float, num_2:float):
    """Sum/Addition: Add two numbers."""
    
    return num_1 + num_2

In [19]:
funcs = [power, sum]
tools = [Tool(func) for func in funcs]

In [20]:
agent = Agent(
    tools=tools
)

we have our agent and now we can invoke:

In [21]:
agent.invoke("what is 2 to the power of 3? then add 11 to the result.")

Terminated loop with message: 'StopReactLoopException'


{'role': 'assistant',
 'content': 'The calculation is complete. If you have any more questions or need further assistance, feel free to ask!',
 'tool_calls': None}

In [22]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI agent, your role is Personal Assistant, and you need to Help users with any questions You can answer multistep questions by sequentially calling functions. You follow a pattern of thought and action. Create a plan of execution: - Use thought to describe your thoughts about the question you've been asked. - Use action to specify one of the tools available to you. When you think its over, call the termination tool. Never try to respond directly if the question requires a tool.The actions you have are the tools: [{'type': 'function', 'function': {'name': 'power', 'description': 'Exponentiation: base to the power of exponent.', 'parallel_tool_calls': False, 'parameters': {'type': 'object', 'properties': {'base': {'type': 'number'}, 'exponent': {'type': 'number'}}, 'required': ['base', 'exponent'], 'additionalProperties': False}, 'strict': True}}, {'type': 'function', 'function': {'name': 'sum', 'description': 'Sum/Addition: Add two numbers.', 

Key Steps in this Agent Creation

1) Environment and Base Classes

    - Required libraries are imported, environment variables are loaded, and the OpenAI client is instantiated
    - Helper classes are prepared
        - Memory class for tracking conversation history
        - Tool class abstraction for wrapping python functions with a JSON schema for tool calling

2) Controlling the ReAct Loop:

    - A custom exception "StopReactLoopException" is defined to gracefully exit the reasoning loop when a task is complete
    - A special "termination" function and correspong message are created so the LLM can explicitly choose to stop

class StopReactLoopException(Exception):
  pass

def call_termination():
  raise StopReactLoopException("Terminating the ReAct loop.")


3) Agent Class Design:

    - The agent class manages the reasoning and tool execution steps

    - Constructor Parameters:
        - name
        - role
        - instructions
        - model
        - temperature
        - tools
    - Core components initialized
        - Memory
        - Tool Map and OpenAI tools
        - Termination Handling
        - system message guiding the thought-action structure


class Agent:
  def __init__(
                self, 
                name="agent", 
                role="personal assistant", 
                instructions=..., 
                model=..., 
                temperature=0.7, 
                tools=[]):
      ...


    - Key Methods

        - invoke(): This is the entry point for users questions, starting the planning loop.
        - react_loop(): This is the main loop that alternates reasoning (plan generation) and action (tool use)
        - reason(): creates a thought and selects an action without tools initially.
        - call_tools(): executes a tool when the model specifies a tool call
        - get_completion(): wraps a standard call to the Openai chat model.
        - register_tool(): adds user-defined tools into the agents toolkit

4) Tool Creation

    - Two tools are created
        -power(base, exponent) - calculates exponentiation
        -sum_numbers(num_1, num_2)

    - These tools are wrapped with JSON schema metadata so the LLM understands their parameters


def power(base: float, exponent: float) -> float:
  return base ** exponent

def sum_numbers(a: float, b: float) -> float:
  return a + b